In [1]:
import os
import pandas as pd
import numpy as np
import sys
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics   



sys.path.append('..')
from Workflow import Workflow

In [2]:
wf = Workflow()
Xtrain, ytrain, Xvalid, yvalid, Xtest, ytest = wf.load_data()

In [23]:
def optimize_rf_model(X, y, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, seed=42):
    estimator = RandomForestRegressor(random_state=seed)  

    # Use GridSearchCV for hyperparameter optimization
    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=cv,
        scoring=scoring,
        n_jobs=n_jobs,
        verbose=3 
    )
    
    # Fit the model to the data
    grid_search.fit(X, y)
    
    # Output the results
    print(f'Best parameters found: {grid_search.best_params_}')
    print(f'Best score: {grid_search.best_score_}')
    
    return grid_search.best_estimator_

In [ ]:
param_grid = {
    'n_estimators': [50, 70, 90, 110], 
    'max_depth': [40, 50, 60],
    'max_features': ['log2'], 
    'min_samples_leaf': [4, 6, 8],
    'min_samples_split': [4, 6, 8],
    'bootstrap': [False], 
}


# Use the optimize_rf_model function to find the best model
seed = 42
best_model = optimize_rf_model(Xtrain, ytrain, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1,seed=seed)

# Evaluate the best model on the training set
y_train_pred = best_model.predict(Xtrain)
train_r2 = metrics.r2_score(ytrain, y_train_pred)

# Evaluate the best model on the validation set
y_valid_pred = best_model.predict(Xvalid)
validation_rmse = np.sqrt(metrics.mean_squared_error(yvalid, y_valid_pred))
validation_r2 = metrics.r2_score(yvalid, y_valid_pred)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(Xtest)
test_r2 = metrics.r2_score(ytest, y_test_pred)

# Print the results
print("Training R2 Score:", train_r2)
print("Validation RMSE:", validation_rmse)
print("Validation R2 Score:", validation_r2)
print("Test R2 Score:", test_r2)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found: {'bootstrap': False, 'max_depth': 50, 'max_features': 'log2', 'min_samples_leaf': 6, 'min_samples_split': 4, 'n_estimators': 90}
Best score: -2766.502380283162
Training R2 Score: 0.846795420949324
Validation RMSE: 53.63609809986627
Validation R2 Score: 0.6994901761583291
Test R2 Score: 0.7346552487906846
